# DotDict class

What I want it to do:
- works with any dictionary whose keys are strings (and whose nested dictionaries keys are strings as well)
- `dotdict_test = DotDict(dict_test)` gives a DotDict object, `dict_test = dict(dotdict_test)` gives back the dictionary
- can access the entries through the "dot" notation AND the regular dictionary index
- setting / accessing / deleting an attribute sets / accesses / deletes the same entry in the dictionary
- can create an empty dotdict with `DotDict()`

initialise:
- `dotdict_test = DotDict(my_dict)`
- `empty_dotdict = DotDict()`

retrieve the dictionary: `my_dict = dict(dotdict_test)`

JSON-like print: `print(dotdict_test)`

`DotDict(eval(str(dotdict_test)))` returns the same DotDict object for dictionaries whose types themselves work with eval

look for key/attribute:
- `hasattr(dotdict.a.b, c)`
- `"c in dotdict.a.b"`
- `"c" in dotdict.a.b.keys()`
- `"c in list(dotdict.a.b)"`
- `"c" in dir(dotdict.a.b)` (`dir` will list `dict` and `DotDict` methods as well)

get:
- `dotdict.a.b.c`
- `dotdict["a"]["b"]["c"]`
- `dotdict.a["b"].c`
- `getattr(dotdict.a.b, c)`

set:
- `dotdict.a.b.c = 0`

delete:
- `dotdict.clear()` empties the DotDict object

TODO:
- setting items is not recursive => OK
- make it work with any type of mapping, not just dicts ? => needs to work recursively => OK
- test deleting
- add a copy method ? => return a shallow copy of DotDict ? => OK
- add pretty prints when called by `print` => OK
- check that every method of `dict` is working as expected
- create new nice template for docstrings

In [1]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)
# dir(IPython.Application.instance().kernel)£

{'status': 'ok', 'restart': True}

In [1]:
# %reset -f
from pathlib import Path
import sys

# ROOT_DIR_PATH = str(Path(__file__).resolve().parent)
ROOT_DIR_PATH = str(Path.cwd().resolve().parent)
if ROOT_DIR_PATH not in sys.path:
    sys.path.insert(0, ROOT_DIR_PATH)

from dotdict import DotDict

## basic tests

In [2]:
# standard dict
test_dict = {'a': {'foo': 'bar'}, 'b': {'lol': 'kekw', 'hehe': 'haha'}, 'c': {}}
dotdict_test = DotDict(test_dict)
print(dotdict_test)

{
    'a': {
        'foo': 'bar'
    },
    'b': {
        'lol': 'kekw',
        'hehe': 'haha'
    },
    'c': {}
}


In [3]:
# empty DotDict, basic insertion
test = DotDict()
print(test)
test.lol = 3
test["haha"] = 6
print(test)

{}
{
    'lol': 3,
    'haha': 6
}


In [4]:
# vars, dict as MappingProxyType objects
vars(test)

mappingproxy({'lol': 3, 'haha': 6})

In [5]:
test.__dict__

mappingproxy({'lol': 3, 'haha': 6})

In [6]:
# _root and _path_to_root accessible but not protected from rewrites
print(test._root)
print(test._path_to_root)

None
None


In [7]:
# test._root = 2
# test["_path_to_root"] = "prout"

In [8]:
# weird DotDict instanciation
test = DotDict(blbl="lol", stuff=74, null_value=None)
print(test)
print(test.null_value)
# print(vars(test))

{
    'blbl': 'lol',
    'stuff': 74,
    'null_value': None
}
None


In [9]:
# another weird DotDict instanciation
test = DotDict({
    "lol": [["foo", "bar"], [1,2]],
    "haha": "abc"
})
print(test)

{
    'lol': [
        [
            'foo',
            'bar'
        ],
        [
            1,
            2
        ]
    ],
    'haha': 'abc'
}


In [10]:
# dict(dotdict_object) returns the dictionary
dict_test_returned = dict(test)
print(type(dict_test_returned["lol"]))
print(dict_test_returned)

<class 'list'>
{'lol': [['foo', 'bar'], [1, 2]], 'haha': 'abc'}


In [11]:
# eval(str(dotdict_object)) returns the dictionary
dict_test_returned = eval(str(test))
print(dict_test_returned)
print(type(dict_test_returned))

{'lol': [['foo', 'bar'], [1, 2]], 'haha': 'abc'}
<class 'dict'>


## init test DotDict objects

In [12]:
dict_test = {
    "param_1": "great",
    "param2": 2,
    # "_lol": "lol",
    # "items": ["a", "b", "c"],
    "param3": {
        "subparam": -5.2,
        # "2": "lol",
        # 1: "haha"
    },
    "param4": [6,1,2]
}
dotdict_test_1 = DotDict(dict_test)
print(dotdict_test_1)

{
    'param_1': 'great',
    'param2': 2,
    'param3': {
        'subparam': -5.2
    },
    'param4': [
        6,
        1,
        2
    ]
}


In [27]:
# replace mappings with dicts when added
dotdict_1 = DotDict({
    "a": 0,
    "b": 1,
    "c": 2
})
dotdict_2 = DotDict({
    "a": 0,
    "b": 1,
    "c": 2
})
dotdict_3 = DotDict({
    "a": {
        
    },
    "b": {
        
    },
    "c": 2
})
dict_dotdict = {
    "first": dotdict_1,
    "second": dotdict_2,
    "third": dotdict_3
}
dotdict_test_2 = DotDict(dict_dotdict)
result_dict = dict(dotdict_test_2)
print(type(result_dict["first"]))
result_dict

<class 'dict'>


{'first': {'a': 0, 'b': 1, 'c': 2},
 'second': {'a': 0, 'b': 1, 'c': 2},
 'third': {'a': {}, 'b': {}, 'c': 2}}

## get values

In [55]:
# accessing values
print(dotdict_test.param3)
print(type(dotdict_test.param3))
print(type(dotdict_test["param3"]))
print(dotdict_test["param4"])
print(hasattr(dotdict_test, "param1"))
print(hasattr(dotdict_test, "fdfd"))
print("param2" in dotdict_test.keys())
print("subparam" in dotdict_test.param3.keys())
print("subparam" in dir(dotdict_test.param3))
print(dotdict_test.param3.subparam)
print(getattr(dotdict_test.param3, "subparam"))

{
    'subparam': -5.2
}
<class 'dotdict.DotDict'>
<class 'dotdict.DotDict'>
[6, 1, 2]
False
False
True
True
True
-5.2
-5.2


## set values

In [18]:
# setting values
dotdict_test.param5 = {1,2,3}
dotdict_test["param6"] = "blblblb"
# dotdict_test["param7"] = DotDict({
dotdict_test["param7"] = {
    "lol": {
        "Lol0": 0,
        # "1": 2,
        # "_lol3": 6,
        # "_ABC": 4
        # 2: "lol"
    },
    "haha": "prout"
}
print(dotdict_test)
setattr(dotdict_test, "param8", "toto")
# dotdict_test["246"] = 4
dotdict_test.param4[0] = -5
print()
print("deep change")
dotdict_test.param7.haha = "awesome"
print("deep change end")
print()
print(dotdict_test.param7.haha)
print()
sub_dotdict = dotdict_test.param7
print(sub_dotdict)
print(type(sub_dotdict))
# print(sub_dotdict._root)
# print(sub_dotdict._path_to_root)
sub_dotdict.foo = "bar"
print(dotdict_test)
print(sub_dotdict)

{
    'param_1': 'great',
    'param2': 2,
    'param3': {
        'subparam': -5.2
    },
    'param4': [
        6,
        1,
        2
    ],
    'param5': {
        1,
        2,
        3
    },
    'param6': 'blblblb',
    'param7': {
        'lol': {
            'Lol0': 0
        },
        'haha': 'prout'
    }
}

deep change
deep change end

awesome

{
    'lol': {
        'Lol0': 0
    },
    'haha': 'awesome'
}
<class 'dotdict.DotDict'>
{
    'param_1': 'great',
    'param2': 2,
    'param3': {
        'subparam': -5.2
    },
    'param4': [
        -5,
        1,
        2
    ],
    'param5': {
        1,
        2,
        3
    },
    'param6': 'blblblb',
    'param7': {
        'lol': {
            'Lol0': 0
        },
        'haha': 'awesome',
        'foo': 'bar'
    },
    'param8': 'toto'
}
{
    'lol': {
        'Lol0': 0
    },
    'haha': 'awesome',
    'foo': 'bar'
}


## delete values

In [22]:
# delete values
delattr(dotdict_test, "param5")
del dotdict_test["param7"]
del dotdict_test.param6
dotdict_test

{'param_1': 'great',
 'param2': 2,
 'param3': {'subparam': -5.2, 'subparam2': -12.6},
 'param4': [-5, 1, 10],
 'param8': 'toto'}

In [30]:
# delete more items
del dotdict_test.third.b
dotdict_test

{'first': {'a': 0, 'b': 1, 'c': 2},
 'second': {'a': 0, 'b': 1, 'c': 2},
 'third': {'a': {}, 'b': {}, 'c': 2}}

## test dict methods

In [19]:
# shallow copy test
dotdict_test_copy = dotdict_test.copy()
# this doesn't change the copy
dotdict_test.param6 = "fgfgf"
# this does
dotdict_test.param4[2] = 10
dotdict_test.param3.subparam2 = -12.6
print(dotdict_test)
print()
print(dotdict_test_copy)
print()
print(type(dotdict_test_copy))

{
    'param_1': 'great',
    'param2': 2,
    'param3': {
        'subparam': -5.2,
        'subparam2': -12.6
    },
    'param4': [
        -5,
        1,
        10
    ],
    'param5': {
        1,
        2,
        3
    },
    'param6': 'fgfgf',
    'param7': {
        'lol': {
            'Lol0': 0
        },
        'haha': 'awesome',
        'foo': 'bar'
    },
    'param8': 'toto'
}

{
    'param_1': 'great',
    'param2': 2,
    'param3': {
        'subparam': -5.2
    },
    'param4': [
        -5,
        1,
        10
    ],
    'param5': {
        1,
        2,
        3
    },
    'param6': 'blblblb',
    'param7': {
        'lol': {
            'Lol0': 0
        },
        'haha': 'awesome',
        'foo': 'bar'
    },
    'param8': 'toto'
}

<class 'dotdict.DotDict'>


In [20]:
vars(dotdict_test)

mappingproxy({'param_1': 'great',
              'param2': 2,
              'param3': {'subparam': -5.2, 'subparam2': -12.6},
              'param4': [-5, 1, 10],
              'param5': {1, 2, 3},
              'param6': 'fgfgf',
              'param7': {'lol': {'Lol0': 0}, 'haha': 'awesome', 'foo': 'bar'},
              'param8': 'toto'})

In [21]:
# update test
dotdict_test.param7.update({"haha": "foo"})
dotdict_test.param7

{'lol': {'Lol0': 0}, 'haha': 'awesome', 'foo': 'bar'}

In [118]:
dict_test = dict(dotdict_test)
dict_test

{'param_1': 'great',
 'param2': 2,
 'param3': {'subparam': -5.2},
 'param4': [-5, 1, 10],
 'param5': {1, 2, 3},
 'param6': 'fgfgf',
 'param7': {'lol': {'Lol0': 0}, 'haha': 'prout'},
 'param8': 'toto'}

In [393]:
# len test
print(len(dict_test), len(dotdict_test))

4 4


In [300]:
"param_1" in dotdict_test

True

In [291]:
dict.update?

Docstring:
D.update([E, ]**F) -> None.  Update D from dict/iterable E and F.
If E is present and has a .keys() method, then does:  for k in E: D[k] = E[k]
If E is present and lacks a .keys() method, then does:  for k, v in E: D[k] = v
In either case, this is followed by: for k in F:  D[k] = F[k]
Type:      method_descriptor

In [235]:
dotdict_test.param7.lol.Lol0

0

In [23]:
dict(dotdict_test)

{'param_1': 'great',
 'param2': 2,
 'param3': {'subparam': -5.2, 'subparam2': -12.6},
 'param4': [-5, 1, 10],
 'param8': 'toto'}

In [28]:
print(type(result_dict["first"]))

<class 'dict'>


In [29]:
print(type(dotdict_test.first))

<class 'dotdict.DotDict'>


## failure cases

In [101]:
dotdict_test.lol

AttributeError: No key/attribute 'lol'

In [173]:
dotdict_test["haha"]

AttributeError: No key/attribute 'haha'

In [174]:
dotdict_test[0] = "lol"

KeyTypeError: The key '0' is not a string

In [176]:
dotdict_test["0"] = "lol"

AttributeNameError: The key '0' cannot be used as an attribute name

In [177]:
del dotdict_test.lol

AttributeError: No key/attribute 'lol'

In [178]:
del dotdict_test["haha"]

AttributeError: No key/attribute 'haha'

In [192]:
# forbidden attribute names
dict_attribute_names = [item for item in dir(dict) if not ("_" in item)]
dict_attribute_names

['clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

# Class inheritance

`super(type[, instance_or_type])`

- `super` returns a proxy item used to call methods from parent or sibling classes
- `super(type)` returns unbound super object
- `super(type, instance)` instance must be instance of type (isinstance(instance, type))
- `super(type, type2)` type2 must be subclass of type (issubclass(type2, type))

`super` is only used when you don't want to name the parent classes by name => behavior determined at rutime using the MRO

In [159]:
%reset -f
class B:
    def __init__(self, b_atr: int):
        print("init B")
        self.b_atr = b_atr
        
    def meth_b(self):
        print("this is a method of the class B")
        
    # def meth(self):
    #     print("this is the method 'meth' of the class A")

# a_test = A(2)
# print(a_test.a_atr)


class C:
    def __init__(self, c_atr: int):
        print("init C")
        self.c_atr = c_atr
        
    def meth_c(self):
        print("this is a method of the class C")
        
    # def meth(self):
    #     print("this is the method 'meth' of the class B")
        

class D(C):
    def __init__(self, c_atr: int, d_atr: int):
        print("init D")
        # print(super())
        # super().__init__(c_atr)
        C.__init__(self, c_atr)
        self.d_atr = d_atr
        print("end init D")
        
    def meth_d(self):
        print("this is a method of the class D")
        
# d_test = D(1,2)
# print(d_test.d_atr)
# print(d_test.c_atr)
# d_test.meth_d()
# d_test.meth_c()


class A(B, D):
    def __init__(self, a_atr: int, b_atr: int, c_atr: int, d_atr: int):
        print("init A")
        B.__init__(self, b_atr)
        D.__init__(self, c_atr, d_atr)
        self.a_atr = a_atr
        print("end init A")
        
    # def super_tests(self):
    #     print(super())
    #     print(super(self.__class__, self))
    #     # print(super(...))

        
# print("A.__class__:", A.__class__)
# print("type(A):", type(A))
# print("MRO:")
for item in A.__mro__:
    print(item)
print()

a_test = A(1,2,3,4)
print(a_test.a_atr)
print(a_test.b_atr)
print(a_test.c_atr)
print(a_test.d_atr)
a_test.meth_b()
a_test.meth_c()
a_test.meth_d()

<class '__main__.A'>
<class '__main__.B'>
<class '__main__.D'>
<class '__main__.C'>
<class 'object'>

init A
init B
init D
init C
end init D
end init A
1
2
3
4
this is a method of the class B
this is a method of the class C
this is a method of the class D


In [12]:
super(A, B)

<super: __main__.A, __main__.B>

In [5]:
super?

Init signature: super(self, /, *args, **kwargs)
Docstring:     
super() -> same as super(__class__, <first argument>)
super(type) -> unbound super object
super(type, obj) -> bound super object; requires isinstance(obj, type)
super(type, type2) -> bound super object; requires issubclass(type2, type)
Typical use to call a cooperative superclass method:
class C(B):
    def meth(self, arg):
        super().meth(arg)
This works for class methods too:
class C(B):
    @classmethod
    def cmeth(cls, arg):
        super().cmeth(arg)
Type:           type
Subclasses:     

In [40]:
%reset -f

class Square:
    
    def __init__(self, side_length):
        self.side_length = side_length
    
    @property
    def area(self):
        return self.side_length ** 2
        
square_test = Square(3)
print(square_test.side_length)
print(square_test.area)


class Cube(Square):
    
    @property
    def area(self):
        return super().area * 6
    
    @property
    def volume(self):
        return self.side_length ** 3

cube_test = Cube(4)
print(cube_test.side_length)
print(cube_test.area)
print(cube_test.volume)

3
9
4
96
64


In [90]:
%reset -f

class Tokenizer:
    """Tokenize text"""
    def __init__(self, text):
        print('Start Tokenizer init')
        self.tokens = text.split()
        print('End Tokenizer init')


class WordCounter(Tokenizer):
    """Count words in text"""
    def __init__(self, text):
        print('Start WordCounter init')
        super().__init__(text)
        self.word_count = len(self.tokens)
        print('End WordCounter init')


class Vocabulary(Tokenizer):
    """Find unique words in text"""
    def __init__(self, text):
        print('Start Vocabulary init')
        super().__init__(text)
        self.vocab = set(self.tokens)
        print('End Vocabulary init')


class TextDescriber(WordCounter, Vocabulary):
    """Describe text with multiple metrics"""
    def __init__(self, text):
        print('Start TextDescriber init')
        super().__init__(text)
        print('End TextDescriber init')
        
        
# for item in TextDescriber.__mro__:
#     print(item)


td = TextDescriber('row row row your boat')
print('--------')
print(td.tokens)
print(td.vocab)
print(td.word_count)

Start TextDescriber init
Start WordCounter init
Start Vocabulary init
Start Tokenizer init
End Tokenizer init
End Vocabulary init
End WordCounter init
End TextDescriber init
--------
['row', 'row', 'row', 'your', 'boat']
{'your', 'boat', 'row'}
5


In [139]:
# parent, child, grandchild:
%reset -f

class Parent:
    def method(self):
        print("parent method")

class Child(Parent):
    def method(self):
        print("child method")
        print(super())
        super().method()

class Grandchild(Child):
    def method(self):
        print("grandchild method")
        print(super())
        super().method()
    
for item in Grandchild.__mro__:
    print(item)
print()

Grandchild().method()

<class '__main__.Grandchild'>
<class '__main__.Child'>
<class '__main__.Parent'>
<class 'object'>

grandchild method
<super: <class 'Grandchild'>, <Grandchild object>>
child method
<super: <class 'Child'>, <Grandchild object>>
parent method


In [141]:
# child and 2 parents:
%reset -f

class Parent1:
    def method(self):
        print("parent 1 method")
        
class Parent2:
    def method(self):
        print("parent 2 method")
        
class Child(Parent1, Parent2):
    def method(self):
        print("child method")
        print(super())
        print(super(Parent1, self))
        # super(Parent1, self).method()
        # super(Parent2, self).method()
        
for item in Child.__mro__:
    print(item)
print()

Child().method()

<class '__main__.Child'>
<class '__main__.Parent1'>
<class '__main__.Parent2'>
<class 'object'>

child method
<super: <class 'Child'>, <Child object>>
<super: <class 'Parent1'>, <Child object>>
parent 2 method


AttributeError: 'super' object has no attribute 'method'

In [135]:
# Child, Parent1, Parent2, Grandparent:
%reset -f

class Grandparent:
    def method(self):
        print("grandparent method")

class Parent1(Grandparent):
    def method(self):
        print("parent1 method")
        print(super())
        super().method()

class Parent2(Grandparent):
    def method(self):
        print("parent2 method")
        print(super())
        super().method()

class Child(Parent1, Parent2):
    def method(self):
        print("child method")
        print(super())
        # print(super().super())
        super().method()
        
for item in Child.__mro__:
    print(item)
print()

Child().method()

<class '__main__.Child'>
<class '__main__.Parent1'>
<class '__main__.Parent2'>
<class '__main__.Grandparent'>
<class 'object'>

child method
<super: <class 'Child'>, <Child object>>
parent1 method
<super: <class 'Parent1'>, <Child object>>
parent2 method
<super: <class 'Parent2'>, <Child object>>
grandparent method


In [94]:
%reset -f

class Parent1:
    def __init__(self):
        print("parent 1 init")
        
class Parent2:
    def __init__(self):
        print("parent 2 init")
        
class Child(Parent1, Parent2):
    def __init__(self):
        print("child init")
        super().__init__()

Child()

child init
parent 1 init


# DUMP

In [53]:
def test(*args, **kwargs):
    print(args)
    print(kwargs)

args_test = (1,2,3)
kwargs_test = {"lol": 0, "lblb": 1}
test(*args_test, **kwargs_test)

(1, 2, 3)
{'lol': 0, 'lblb': 1}


In [2]:
pattern = re.compile("^[a-zA-Z]\w*$")
pattern

re.compile(r'^[a-zA-Z]\w*$', re.UNICODE)

In [3]:
test_list = [
    "MyClass",
    "_a_hidden_parametter",
    "__dunder__",
    "param_6",
    "YikeS_23",
    "aLBERT2dse",
    "0",
    "25laurence",
    "Myclass#6",
    "This doesn't work either",
    "test's"
]
for str_ in test_list:
    if pattern.match(str_):
        print("match")
    else:
        print("no match")

match
no match
no match
match
match
match
no match
no match
no match
no match
no match


In [90]:
class Test:
    pass
test_dir_return = Test().__dir__()
test_dir_return

['__module__',
 '__dict__',
 '__weakref__',
 '__doc__',
 '__new__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__init__',
 '__reduce_ex__',
 '__reduce__',
 '__getstate__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [91]:
type(test_dir_return)

list

In [205]:
class NullTest:
    def __init__(self):
        self.lol = "kaka"
        print(self.lol)
        # self = None


null_test = NullTest()
null_test.__dict__

kaka


{'lol': 'kaka'}

In [332]:
DotDict({'one': 1, 'three': 3}, two=2)

{'one': 1, 'three': 3, 'two': 2}

In [363]:
test = ["a", "l", "z"]
test.append("e")
test

['a', 'l', 'z', 'e']

In [ ]:
dir(object)

In [ ]:
list(vars(object).keys())

In [25]:
len(vars(object).keys())

24

In [26]:
len(dir(object))

24

In [ ]:
dir(dict)

In [ ]:
dir(dotdict_test)

In [ ]:
vars(dict)

In [52]:
dict_test = {"lol": "haha"}
vars(dict_test)

TypeError: vars() argument must have __dict__ attribute

In [53]:
vars(dotdict_test)

{}

In [54]:
vars(DotDict)

mappingproxy({'__module__': '__main__',
              '__doc__': '\n    Allows handling dictionaries with the "dot" notation.\n    ',
              '__init__': <function __main__.DotDict.__init__(self, *args, _check: bool = True, **kwargs)>,
              '__getitem__': <function __main__.DotDict.__getitem__(self, key: str) -> Optional[Any]>,
              '__getattr__': <function __main__.DotDict.__getitem__(self, key: str) -> Optional[Any]>,
              '__setitem__': <function __main__.DotDict.__setitem__(self, key: str, value: Optional[Any])>,
              '__setattr__': <function __main__.DotDict.__setitem__(self, key: str, value: Optional[Any])>,
              '__delitem__': <function __main__.DotDict.__delitem__(self, key: str)>,
              '__delattr__': <function __main__.DotDict.__delitem__(self, key: str)>,
              '__dir__': <function __main__.DotDict.__dir__(self) -> list[str]>,
              '__dict__': <attribute '__dict__' of 'DotDict' objects>,
            

In [56]:
list_test = [1,2,3]
vars(list_test)

TypeError: vars() argument must have __dict__ attribute

In [17]:
class Test:
    pass

object_test = Test()
# dir(object_test)

In [18]:
vars(object_test)

{}

In [20]:
object_test.b = 2
object_test.b

2

In [131]:
object_test.__dict__["b"] = 3
object_test.__dict__["c"] = 18
print(object_test.b, object_test.c)

3 18


In [132]:
vars(object_test)["d"] = -5
object_test.d

-5

In [133]:
# delete an attribute with vars and __dict___ ?
del vars(object_test)["d"]
del object_test.__dict__["c"]
object_test.c

AttributeError: 'Test' object has no attribute 'c'

In [124]:
test = [1,2,3]
test

[1, 2, 3]

In [125]:
copy_test = test.copy()
copy_test

[1, 2, 3]

In [126]:
copy_test.append(4)
copy_test

[1, 2, 3, 4]

In [127]:
test

[1, 2, 3]

In [130]:
copy_test = test + [4]
copy_test

[1, 2, 3, 4]

In [131]:
test

[1, 2, 3]

In [153]:
class Test:
    def __setitem__(self, key, value):
        print("__setitem__ call")
        super().__setitem__(key, value)
    def __setattr__(self, key, value):
        print("__setattr__ call")
        super().__setattr__(key, value)

test = Test()
test.a = "b"
test.a

__setattr__ call


'b'

In [3]:
MappingProxyType?

Init signature: MappingProxyType(self, /, *args, **kwargs)
Docstring:      <no docstring>
Type:           type
Subclasses:     

In [23]:
test = object()
test.lol = 2
test

AttributeError: 'object' object has no attribute 'lol'

In [26]:
test.__dict__ = {}

AttributeError: 'object' object has no attribute '__dict__'

In [27]:
test.__setattr__("__dict__", {})
test

AttributeError: 'object' object has no attribute '__dict__'

In [28]:
test.__setattr__

<method-wrapper '__setattr__' of object object at 0x000002214C6FA3E0>

In [24]:
dict.__str__

<slot wrapper '__str__' of 'object' objects>

In [64]:
test = {"lol": 2}
test_it = iter(test)

In [65]:
test = "lol"
test_it = iter(test)

In [68]:
test = (1,2,3)
iter(test)

In [66]:
next(test_it)

'l'

In [22]:
[e for e in "lol"]

['l', 'o', 'l']

In [24]:
"lol"[1]

'o'

In [16]:
test = [1,2,3]
test_it = iter(test)
next(test_it)

1

In [12]:
test = {0,1,2}
iter(test)

In [17]:
a = 0
try:
    a = 1/0
except:
    pass
a

0

In [47]:
from collections.abc import Mapping, Iterable, Sequence, MutableSequence
from typing import Any
# isinstance(DotDict(), Mapping)
isinstance("lol", MutableSequence)

False

In [9]:
isinstance(vars(DotDict()), Mapping)

True

In [8]:
isinstance("abc", Mapping)

False

In [48]:
isinstance((1,2,3), MutableSequence)

False

In [49]:
iter(range(0,10))

In [13]:
dir(object())

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [14]:
dir(dict())

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [31]:
test = b"still allows embedded 'single' quotes"
iter(test)

In [32]:
type(test) is str

False

In [33]:
iter(1845)

TypeError: 'int' object is not iterable

In [35]:
test = bytearray(b'Hi!')
iter(test)

In [69]:
repr((1,2,3))

'(1, 2, 3)'

In [72]:
str({1,2,3})

'{1, 2, 3}'

In [56]:
filter?

Init signature: filter(self, /, *args, **kwargs)
Docstring:     
filter(function or None, iterable) --> filter object

Return an iterator yielding those items of iterable for which function(item)
is true. If function is None, return the items that are true.
Type:           type
Subclasses:     

In [64]:
test = list(filter(None, [3,0,1,2]))
test

[3, 1, 2]

In [78]:
# sum(), any(), all(), max(), min(), and len()
from functools import reduce

_len = lambda it: reduce(
    lambda value, element: value + 1,
    it,
    0
)

_sum = lambda it: reduce(
    lambda value, element: value + element,
    it
)

_any = lambda it: reduce(
    lambda value, element: bool(value) or bool(element),
    it,
    False
)

_all = lambda it: reduce(
    lambda value, element: bool(value) and bool(element),
    it,
    True
)

_max = lambda it: reduce(
    lambda value, element: element if element > value else value,
    it
)

_min = lambda it: reduce(
    lambda value, element: element if element < value else value,
    it
)

print(_len(["a", "b", "c"]))
print(_sum([3,1,2]))
print(_any([False, False]))
print(_any([False, True]))
print(_all([True, True]))
print(_all([False, True]))
print(_max([1,5,-2]))
print(_min([1,5,-2]))

3
6
False
True
True
False
5
-2


In [68]:
test_lamda = lambda x, y: x + y
test_lamda(2,3)

5

In [6]:
def title_printer(title: str, fill_char: str = "#", print_: bool = True) -> str:
    """
    ###########################
    ### super awesome title ###
    ###########################
    """
    # create title string
    title_lines = []
    title_lines.append("####" + "#"*len(title) + "####")
    title_lines.append("### " + title + " ###")
    title_lines.append("####" + "#"*len(title) + "####")
    title_string = "\n".join(title_lines)
    # print it and return it
    if print_:
        print(title_string)
    return title_string

print(title_printer("super awesome title", print_=False))

###########################
### super awesome title ###
###########################


In [9]:
print(title_printer("title printer", print_=False))

#####################
### title printer ###
#####################
